### **Decorators**

Decoraror is just a function that takes a function as an arguement alters it's functionality and return another function without altering the source code of the original function you passed in. So basically, A decorator wraps a function and returns it's behaviour.

Let's look at some quick examples.

In [2]:
def decorator_func(func):
    def wrapper():
        print("Something was done before func is called")
        func()
        print("Something happened after the function was called")
    return wrapper

In [3]:
def say_cheese():
    print("Cheese!")

modified_say_cheese = decorator_func(say_cheese)
print(modified_say_cheese)

<function decorator_func.<locals>.wrapper at 0x00000284AEF056C0>


In [4]:
modified_say_cheese()

Something was done before func is called
Cheese!
Something happened after the function was called


Okay we see how decorators work. *Why would we even what to do this ?*



Decorating our functions allows us to add functionality to our existing funtions while adding that functionality  inside our wrapper, hence not modifying our existing function (or having to restructure our existing function) because we might need it later on in our code.

**Adding syntatic Sugar (like writing fancy code)**

instead of writng:
```python
 decorator_func(say_cheese)
```
we can write our decorator like this: 
```python
 @decorator_func
```
on top of our function function

In [5]:
@decorator_func
def say_cheese():
    print("cheese!")


this way any time I call `say_cheese` it will have the decorator functionality

In [6]:
say_cheese()

Something was done before func is called
cheese!
Something happened after the function was called


**Tip:** *A good practice is to write all your decorator functions in a file and import them.* 

Now, If we write a decorator function `decorator_func` specifically for a certain function, It will only work with functions of similar signature (i.e arguement type, no of arguements and structure)

To demonstrate this we will write a decorator to log some information about our code, in this case arguements.

In [7]:
def log_arguements(func):
    def wrapper(a, b):
        print(f"Function {func.__name__} was called with arguements: {a}, {b}")
        return func(a, b)
    return wrapper

In [8]:
@log_arguements
def add(a, b):
    return a + b

add(3, 4)

Function add was called with arguements: 3, 4


7

Now let's use the decorator again with another function with a similar functionality and see what happens

In [9]:
@log_arguements
def add(a, b):
    return a + b

add(3, 4)

@log_arguements
def product(a, b):
    return a * b

product(3, 4)

Function add was called with arguements: 3, 4
Function product was called with arguements: 3, 4


12

let's try to use this decorator to log a function that takes two arguements one `keyword` the other is just a regular argument. let's see what happens:

In [10]:
@log_arguements
def introduction(name, age= 23):
    return f"Hi, my name is {name} and I am {age} years old "


In [11]:
introduction("John", age = 30)

TypeError: log_arguements.<locals>.wrapper() got an unexpected keyword argument 'age'

we get an error, because we passed in a keyword arguement, and our decorator was not designed to work with key word arguements

In [ ]:
introduction("John", 24)

Function introduction was called with arguements: John, 24


'Hi, my name is John and I am 24 years old '

The above will work because age was'nt passed a keyword arguement.

In other to avoid this confusion we use the **`*arg`** and **`**kwargs`** in our wrapper in the decorator function and our function so the decorator can handle anytype of input

In [11]:
def log_arguements(func):
    def wrapper(*args, **kwargs):
        print(f"Function {func.__name__} was called with arguements: {args}, {kwargs}")
        return func(*args, **kwargs)
    return wrapper

In [12]:
@log_arguements
def introduction(name, age= 23):
    return f"Hi, my name is {name} and I am {age} years old "


In [13]:
introduction("John", age = 30)

Function introduction was called with arguements: ('John',), {'age': 30}


'Hi, my name is John and I am 30 years old '

### **Writing a Decorator with a Class**

So basically when we want to use a class as a blueprint for decorators (i.e instantiating decorators), we usually pass our function to the `__init__()` method (if we read OOP notebook the reason for this is very obvious... lol... so basicall it's so we can instantiate a decorator object for a specified function as we know upon instantiating the object python runs the `__init__()` method automatically)  and our wrapper function is defined as the callable method `__call__()`.

In [30]:
class Decorator(object):
    def __init__(self, func):
        self.func  = func
    
    def __call__(self, *args, **kwargs):
        print(f"Function {self.func.__name__} was called with arguements: {args}, {kwargs}")
        return self.func(*args, **kwargs)
        

In [31]:
@Decorator
def introduction(name, age= 23):
    return f"Hi, my name is {name} and I am {age} years old "


In [32]:
introduction("John", age = 30)

Function introduction was called with arguements: ('John',), {'age': 30}


'Hi, my name is John and I am 30 years old '

We can see classes can be used used to write decorators. some people use classes to write  decorators so as to levergae the capabilities of classes in their decorators but in most cases decorators are writing with functions.

### **Forgetting who you are and finding yourself**

we already know that the decorator takes our function and adds some functionality to it, essentially by using a warapper function and the decorator returns the wrapper function. 

```python
@decorator
``` 
is the same as:
```python
decorator(func) 
```
and then we know that the structure of our generator is:
```python
def decorator(func):
    def wrapper(*args, **kwargs):
       ....add some logic around func....
       return the result of our optimisation on func
    return wrapper       

```

The implication of this is that whenever we use decorate our function with this decorator the decorated function returns `wrapper` consequently making our decorator function think it's name is wrapper.

In [17]:
introduction.__name__

'wrapper'

In [18]:
help(introduction)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



Well this is not harmful to us, but it can a pain in the ass to someone trying to study or test our code especially when we are chaining / stacking multiple decorators on a function. for example:

```python
@logger
@timer

def func():
    .....
```
if the logger function is logging a message containing the name of then function `func` and it's arguement. instead of seeing `func` in our logs we would be seeing the `wrapper` for timer decorator which is not good. 

In [23]:
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"Function {func.__name__} was called with arguements: {args}, {kwargs}")
        return func(*args, **kwargs)
    return wrapper

def timer(func):
    import time
    def time_wrapper(*args, **kwargs):
        start =  time.time()
        time.sleep(1) # delay time by 1 second
        func(*args, **kwargs)
        stop =  time.time()
        total_time = stop -  start
        return f"{func.__name__}() ran in {total_time:.4f} seconds"
    return time_wrapper

In [24]:
@logger
@timer
def greet(name, age):
    print(f"Hi, {name} is {age} years old.")

In [26]:
greet("James", age=25)

Function time_wrapper was called with arguements: ('James',), {'age': 25}
Hi, James is 25 years old.


'greet() ran in 1.0012 seconds'

we can clearly see that the logged output says `time_wrapper` insted of `greet` this output can be confusing to someone trying to debug or understand our code.  

this is because when we used multiple decorators 
we chained them:
```python
@logger
@timer
def func....
```
this is the same as 
```python
logger(timer(func))
```

and the timmer decorator return it's wrapper which the logger took and logged.


To Fix this we would use the **`@functools.wraps`** decorator before our wrappers.

In [27]:
import functools
def logger(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Function {func.__name__} was called with arguements: {args}, {kwargs}")
        return func(*args, **kwargs)
    return wrapper

def timer(func):
    import time
    @functools.wraps(func)
    def time_wrapper(*args, **kwargs):
        start =  time.time()
        time.sleep(1) # delay time by 1 second
        func(*args, **kwargs)
        stop =  time.time()
        total_time = stop -  start
        return f"{func.__name__}() ran in {total_time:.4f} seconds"
    return time_wrapper

In [28]:
@logger
@timer
def greet(name, age):
    print(f"Hi, {name} is {age} years old.")

In [29]:
greet("James", age=25)

Function greet was called with arguements: ('James',), {'age': 25}
Hi, James is 25 years old.


'greet() ran in 1.0013 seconds'

we can see now that this is much better. so when ever we are writing decorators that would be chained together we should always add **`@functools.wraps`** decorator before our wrapper code.